# Computer Vision: Gap Calculation using YOLO net and OpenCV

In [46]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import Counter
import time as pytime
import pandas as pd
import sys
import os
from datetime import datetime

In [47]:
# Use GPU if available
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Load the Anti-Alpine optimized model
model = YOLO("weights/fine_tuned.pt")
model.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

---

## 1. Gap Calculation and Yolo video processing

### 1.1 Dedining constants and variables

In [48]:
# Size and scale configuration
FRAME_WIDTH = 1280  # Higher resolution for better detail
CAR_LENGTH_METERS = 5.63  # Real car length in meters

# FOR GAPS: use a lower global threshold to maximize detections
GAP_DETECTION_THRESHOLD = 0.40  # Low threshold to detect all possible cars

# Specific colors for each F1 team (BGR format for OpenCV)
class_colors = {
    'Ferrari': (0, 0, 255),         # Red (BGR)
    'Mercedes': (200, 200, 200),    # Silver (BGR)
    'Red Bull': (139, 0, 0),        # Dark Blue (BGR)
    'McLaren': (0, 165, 255),       # Orange (BGR)
    'Aston Martin': (0, 128, 0),    # Green (BGR)
    'Alpine': (128, 0, 0),          # Blue (BGR)
    'Williams': (205, 0, 0),        # Light Blue (BGR)
    'Haas': (255, 255, 255),        # White (BGR)
    'Kick Sauber': (255, 255, 0),   # Cyan (BGR)
    'Racing Bulls': (0, 0, 255),    # Red (BGR)
    'background': (128, 128, 128),  # Gray (BGR)
    'unknown': (0, 255, 255)        # Yellow for cars without secure classification
}

# Thresholds to show classification (not for filtering detections)

class_thresholds = {
    'Kick Sauber': 0.55, 
    'Red Bull': 0.85,     
    'Alpine': 0.85,       
    'Ferrari': 0.45,       
    'Haas': 0.40,          
    'McLaren': 0.60,      
    'Mercedes': 0.40,      
    'Williams': 0.75,     
    'background': 0.60    
}

# Detection history for stabilization
last_detections = {}
track_history = {}
id_counter = 0
class_history = {}

### 1.2 Calculating the gap

In [49]:
def calculate_gap(box1, box2, class1, class2):
    """Calculates the distance between centers using car width for scale"""
    # Box centers
    cx1, cy1 = (box1[0] + box1[2])/2, (box1[1] + box1[3])/2
    cx2, cy2 = (box2[0] + box2[2])/2, (box2[1] + box2[3])/2
    
    # Distance in pixels
    pixel_distance = np.hypot(cx2 - cx1, cy2 - cy1)
    
    # Scale based on average width of detected cars
    avg_width = ((box1[2] - box1[0]) + (box2[2] - box2[0])) / 2
    scale = CAR_LENGTH_METERS / avg_width if avg_width != 0 else 0
    
    # Calculate gap time at 300km/h (83.33 m/s)
    speed_mps = 83.33  # Meters per second at 300km/h
    gap_time = (pixel_distance * scale) / speed_mps
    
    return {
        'distance': pixel_distance * scale,  # Distance in meters
        'time': gap_time,                   # Time in seconds at 300km/h
        'car1': class1,                     # Team of first car
        'car2': class2                      # Team of second car
    }


### 1.3 Processing the video with YOLO

In [50]:
def process_video_with_yolo(video_path, output_path=None):
    global last_detections, track_history, id_counter, class_history, GAP_DETECTION_THRESHOLD, track_age
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return
    
    # Get original video dimensions
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    current_frame = 0
    
    # Calculate new height maintaining aspect ratio
    target_height = int(FRAME_WIDTH * original_height / original_width)
    
    out = None
    if output_path:
        # Change codec from 'mp4v' to 'XVID' which is more reliable
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (FRAME_WIDTH, target_height))
        if not out.isOpened():
            print(f"Error: Could not create output video file at {output_path}")
            print("Continuing without saving output...")
            output_path = None
    
    # Variables for calculating real FPS
    frame_count = 0
    start_time = pytime.time()
    current_fps = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: 
            break
        
        frame_count += 1
        current_frame += 1
        
        # Resize maintaining aspect ratio
        frame_resized = cv2.resize(frame, (FRAME_WIDTH, target_height))
        original_frame = frame_resized.copy()
        
        # Run YOLOv8 detection with optimized threshold for YOLOv12m fine-tuned model
        results = model.predict(
            source=frame_resized, 
            conf=GAP_DETECTION_THRESHOLD,  # Adjusted threshold (0.40 recommended)
            iou=0.45,   # Standard IoU
            max_det=20, # Maximum detections
            verbose=False
        )[0]
        
        # Current detections
        current_detections = {}
        
        # Process detection results
        if results.boxes and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()
            class_ids = results.boxes.cls.cpu().numpy().astype(int)
            
            # Create detection list with all information
            detections = []
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                conf = float(confs[i])
                class_id = int(class_ids[i])
                cls_name = model.names[class_id]
                
                # Determine whether to show team classification based on threshold
                classified = conf >= class_thresholds.get(cls_name, 0.40)
                
                # Calculate center and size metrics
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                area = (x2 - x1) * (y2 - y1)
                aspect_ratio = (x2 - x1) / (y2 - y1) if (y2 - y1) > 0 else 0
                
                # Filter by size/proportion for problematic classes
                is_reasonable_size = area > (FRAME_WIDTH * target_height * 0.003)
                is_reasonable_ratio = 0.4 < aspect_ratio < 2.5
                
                # Special filtering for Alpine and Red Bull
                if (cls_name == 'Alpine' or cls_name == 'Red Bull'):
                    if not (is_reasonable_size and is_reasonable_ratio and conf > class_thresholds.get(cls_name, 0.60)):
                        classified = False
                        cls_name = "F1 Car"  # Generic
                
                # Assign unique ID or retrieve existing ID
                object_id = None
                for old_id, old_info in last_detections.items():
                    old_cx, old_cy = old_info['center']
                    old_cls = old_info['class']
                    
                    # Distance between centers
                    dist = np.sqrt((center_x - old_cx)**2 + (center_y - old_cy)**2)
                    
                    # If it's close, it could be the same object
                    if dist < 100:
                        object_id = old_id
                        
                        # If previous class was valid and new one is uncertain, keep the previous one
                        if old_info['classified'] and not classified:
                            cls_name = old_cls
                            classified = True
                        
                        # Enhanced stabilization for problematic classes
                        if classified and old_cls != cls_name:
                            # Alpine has high mAP but low precision (0.447)
                            if cls_name == 'Alpine' and conf < 0.65:
                                if old_info['classified']:
                                    cls_name = old_cls
                            # McLaren has low precision (0.378)
                            elif cls_name == 'McLaren' and conf < 0.55:
                                if old_info['classified']:
                                    cls_name = old_cls
                            # Williams has good mAP50-95 (0.651) but classification problems
                            elif cls_name == 'Williams':
                                if old_info['classified'] and old_cls != 'Williams':
                                    cls_name = old_cls
                        break
                
                # If no match found, assign new ID
                if object_id is None:
                    object_id = id_counter
                    id_counter += 1
                    track_history[object_id] = []
                    class_history[object_id] = []
                    track_age[object_id] = 0
                else:
                    # Update track age for existing objects
                    track_age[object_id] += 1
                
                # If track is new (first 3 frames), be more strict with Alpine/Red Bull
                if track_age[object_id] < 3 and (cls_name == 'Alpine' or cls_name == 'Red Bull') and conf < 0.75:
                    classified = False
                    cls_name = "F1 Car"  # Use generic to avoid early false classifications
                
                # Update history
                if object_id in class_history:
                    # Only add to history if we're sure of the class
                    if classified:
                        class_history[object_id].append(cls_name)
                        # Limit history to 5 classes
                        if len(class_history[object_id]) > 5:
                            class_history[object_id].pop(0)
                    
                    # Use the most common class from history for stability
                    if len(class_history[object_id]) >= 5:  # Increased from 3 to 5
                        counts = Counter(class_history[object_id])
                        if counts:  # Make sure it's not empty
                            most_common = counts.most_common(1)[0][0]
                            cls_name = most_common
                            classified = True
                
                # Save current detection
                current_detections[object_id] = {
                    'box': (x1, y1, x2, y2),
                    'conf': conf,
                    'class': cls_name,
                    'classified': classified,
                    'center': (center_x, center_y),
                    'area': area,
                    'y_bottom': y2  # For sorting by vertical position
                }
                
                # Add to detection list for gap calculation
                detections.append({
                    'id': object_id,
                    'box': (x1, y1, x2, y2),
                    'class': cls_name,
                    'classified': classified,
                    'conf': conf,
                    'y_bottom': y2
                })
            
            # Sort by vertical position (cars more below first - closer)
            detections = sorted(detections, key=lambda x: x['y_bottom'], reverse=True)
            
            # Eliminate ghost detections
            detections_to_keep = []
            for det in detections:
                # If it's Alpine, verify if there are other cars very close (possible false positive)
                if det['class'] == 'Alpine' and det['classified']:
                    is_ghost = False
                    
                    for other in detections:
                        if other['id'] != det['id']:
                            # Calculate simple overlap between boxes
                            x1a, y1a, x2a, y2a = det['box']
                            x1b, y1b, x2b, y2b = other['box']
                            
                            overlap = max(0, min(x2a, x2b) - max(x1a, x1b)) * max(0, min(y2a, y2b) - max(y1a, y1b))
                            area_a = (x2a - x1a) * (y2a - y1a)
                            
                            # If there's significant overlap, likely a false positive
                            if overlap > 0.3 * area_a:
                                is_ghost = True
                                break
                    
                    if not is_ghost:
                        detections_to_keep.append(det)
                else:
                    detections_to_keep.append(det)
                    
            detections = detections_to_keep
            
            # Draw boxes and gaps
            for i, det in enumerate(detections):
                x1, y1, x2, y2 = det['box']
                cls_name = det['class']
                conf = det['conf']
                classified = det['classified']
                
                # Get specific color for the team
                if classified:
                    color = class_colors.get(cls_name, (0, 255, 0))
                else:
                    color = class_colors['unknown']  # Yellow for cars without secure classification
                
                # Draw box with team color
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                
                # Label with class and confidence
                if classified:
                    label = f"{cls_name}: {conf:.2f}"
                else:
                    label = f"F1 Car: {conf:.2f}"
                
                t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                cv2.rectangle(frame_resized, (x1, y1-t_size[1]-3), (x1+t_size[0], y1), color, -1)
                cv2.putText(frame_resized, label, (x1, y1-3), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Only if there's a next car
                if i < len(detections)-1:
                    next_det = detections[i+1]
                    gap_info = calculate_gap(
                        det['box'], next_det['box'], 
                        det['class'] if det['classified'] else "F1 Car", 
                        next_det['class'] if next_det['classified'] else "F1 Car"
                    )
                    
                    # Connection points
                    cx1, cy1 = int((x1+x2)/2), int(y1)  # Use top of the car
                    nx1, ny1, nx2, ny2 = next_det['box']
                    cx2, cy2 = int((nx1+nx2)/2), int(ny2)  # Use bottom of the next car
                    
                    # Diagonal line between cars
                    cv2.line(frame_resized, (cx1, cy1), (cx2, cy2), (0, 255, 0), 2)
                    
                    # Text at midpoint with more information
                    mid_x, mid_y = (cx1+cx2)//2, (cy1+cy2)//2
                    
                    # Distance and gap time
                    dist_text = f"{gap_info['distance']:.1f}m"
                    time_text = f"{gap_info['time']:.2f}s"
                    
                    # Background for text
                    dist_size = cv2.getTextSize(dist_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    time_size = cv2.getTextSize(time_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    
                    # Draw semi-transparent background
                    overlay = frame_resized.copy()
                    cv2.rectangle(overlay, 
                                 (mid_x - 5, mid_y - 50), 
                                 (mid_x + max(dist_size[0], time_size[0]) + 10, mid_y + 10),
                                 (0, 0, 0), -1)
                    cv2.addWeighted(overlay, 0.6, frame_resized, 0.4, 0, frame_resized)
                    
                    # Draw texts
                    cv2.putText(frame_resized, dist_text, (mid_x, mid_y - 25),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    cv2.putText(frame_resized, time_text, (mid_x, mid_y),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Update last_detections for next iteration
        last_detections = current_detections
        
        # Calculate FPS
        if frame_count % 10 == 0:
            current_time = pytime.time()
            current_fps = 10.0 / (current_time - start_time)
            start_time = current_time
        
        # Show FPS and model information
        cv2.putText(frame_resized, f"FPS: {current_fps:.1f}", (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.putText(frame_resized, "F1 Gap Detection", (FRAME_WIDTH - 300, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Show detection mode and progress
        detection_mode = f"Detection Threshold: {GAP_DETECTION_THRESHOLD:.2f}"
        cv2.putText(frame_resized, detection_mode, (20, target_height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Show video progress
        progress_text = f"Frame: {current_frame}/{total_frames} ({current_frame/total_frames*100:.1f}%)"
        cv2.putText(frame_resized, progress_text, (FRAME_WIDTH - 400, target_height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Save processed frame if requested
        if output_path:
            out.write(frame_resized)
        
        # Show frame
        cv2.imshow("F1 Gap Detection", frame_resized)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('+'):  # Increase threshold
            GAP_DETECTION_THRESHOLD = min(GAP_DETECTION_THRESHOLD + 0.05, 0.95)
            print(f"Detection threshold increased to {GAP_DETECTION_THRESHOLD:.2f}")
        elif key == ord('-'):  # Decrease threshold
            GAP_DETECTION_THRESHOLD = max(GAP_DETECTION_THRESHOLD - 0.05, 0.05)
            print(f"Detection threshold decreased to {GAP_DETECTION_THRESHOLD:.2f}")
        elif key == ord('d'):  # Skip forward 10 seconds
            skip_frames = int(fps * 10)  # 10 seconds * fps = number of frames to skip
            new_frame_pos = min(current_frame + skip_frames, total_frames - 1)
            cap.set(cv2.CAP_PROP_POS_FRAMES, new_frame_pos)
            current_frame = new_frame_pos - 1  # Will be incremented in the next cycle
            # Temporarily reset tracking
            last_detections = {}
            print(f"Skipped forward 10 seconds to frame {new_frame_pos}")
    
    # Release resources
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

### 1.4 Running a demo 

In [51]:
#Run with your video
def main():
    # video_path = "../f1-strategy/data/videos/best_overtakes_2023.mp4.f399.mp4"
    # video_path = "../f1-strategy/data/videos/spain_2023_race.mp4.f399.mp4"
    
    video_path = "../f1-strategy/data/videos/belgium_gp.f399.mp4"
    output_path = "../f1-strategy/data/videos/gap_detection_output.mp4"
    process_video_with_yolo(video_path, output_path)

if __name__ == "__main__":
    main()

Error opening video: ../f1-strategy/data/videos/belgium_gp.f399.mp4


Controls

'q': out
'+': more detection threshold
'-': less detection threshold
'd': 10 seconds ahead 

---

## 2. Gap Extraction 

In [52]:
def extract_gaps_from_video(video_path, sample_interval_seconds=10, output_csv=None, show_video=True):
    """
    Process a video and extract gap data at regular intervals with visualization
    
    Args:
        video_path: Path to the F1 video
        sample_interval_seconds: How often to sample gap data (in seconds)
        output_csv: Path to save CSV data (if None, will generate a default path)
        show_video: Whether to display the video during processing
    
    Returns:
        DataFrame with extracted gap data
    """
    global last_detections, track_history, id_counter, class_history, GAP_DETECTION_THRESHOLD, track_age
    
    # Initialize empty list to store all gap data
    all_gaps = []
    
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return None
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate frame interval based on desired time interval
    frame_interval = int(fps * sample_interval_seconds)
    print(f"Video FPS: {fps}, sampling every {frame_interval} frames ({sample_interval_seconds} seconds)")
    
    # Initialize tracking variables
    frame_count = 0
    current_frame = 0
    last_sample_frame = -frame_interval  # Ensure we get a sample on first frame
    start_time = pytime.time()
    current_fps = 0
    
    # Reset tracking variables
    last_detections = {}
    track_history = {}
    id_counter = 0
    class_history = {}
    track_age = {}
    
    # Size configuration
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    target_height = int(FRAME_WIDTH * original_height / original_width)
    
    print(f"Starting gap extraction from {video_path}...")
    print(f"Will sample approximately every {sample_interval_seconds} seconds")
    
    # Process video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        current_frame += 1
        
        # Calculate video timestamp in seconds
        timestamp = current_frame / fps
        
        # Determine if this is a sample frame
        is_sample_frame = (current_frame - last_sample_frame >= frame_interval)
        
        # Skip processing if not showing video and not a sample frame
        if not show_video and not is_sample_frame:
            if current_frame % 100 == 0:  # Show progress occasionally
                print(f"Progress: Frame {current_frame}/{total_frames} ({current_frame/total_frames*100:.1f}%)")
            continue
        
        # Mark this as a sample frame if needed
        if is_sample_frame:
            last_sample_frame = current_frame
            print(f"Taking sample at frame {current_frame} (timestamp: {timestamp:.2f}s)")
        
        # Resize frame for processing
        frame_resized = cv2.resize(frame, (FRAME_WIDTH, target_height))
        original_frame = frame_resized.copy()
        
        # Run YOLOv8 detection with optimized threshold
        results = model.predict(
            source=frame_resized, 
            conf=GAP_DETECTION_THRESHOLD,  # Using updated threshold (0.40 recommended)
            iou=0.45,
            max_det=20,
            verbose=False
        )[0]
        
        # Current detections dictionary
        current_detections = {}
        
        # Process detection results
        if results.boxes and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()
            class_ids = results.boxes.cls.cpu().numpy().astype(int)
            
            # Create detection list
            detections = []
            
            # Process each detected object
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                conf = float(confs[i])
                class_id = int(class_ids[i])
                cls_name = model.names[class_id]
                
                # Check if confidence meets threshold
                classified = conf >= class_thresholds.get(cls_name, 0.40)
                
                # Calculate size metrics for filtering
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                area = (x2 - x1) * (y2 - y1)
                aspect_ratio = (x2 - x1) / (y2 - y1) if (y2 - y1) > 0 else 0
                
                # Filter by size/proportion for problematic classes
                is_reasonable_size = area > (FRAME_WIDTH * target_height * 0.003)
                is_reasonable_ratio = 0.4 < aspect_ratio < 2.5
                
                # Special filtering for Alpine and Red Bull
                if (cls_name == 'Alpine' or cls_name == 'Red Bull'):
                    if not (is_reasonable_size and is_reasonable_ratio and conf > class_thresholds.get(cls_name, 0.60)):
                        classified = False
                        cls_name = "F1 Car"  # Generic classification
                
                # Try to match with existing objects for tracking
                object_id = None
                for old_id, old_info in last_detections.items():
                    old_cx, old_cy = old_info['center']
                    old_cls = old_info['class']
                    
                    # Calculate distance between centers
                    dist = np.sqrt((center_x - old_cx)**2 + (center_y - old_cy)**2)
                    
                    # If close enough, it's likely the same car
                    if dist < 100:
                        object_id = old_id
                        
                        # Use previous classification if it was better
                        if old_info['classified'] and not classified:
                            cls_name = old_cls
                            classified = True
                        
                        # Enhanced stabilization for problematic classes
                        if classified and old_cls != cls_name:
                            # Alpine has high mAP but low precision (0.447)
                            if cls_name == 'Alpine' and conf < 0.65:
                                if old_info['classified']:
                                    cls_name = old_cls
                            # McLaren has low precision (0.378)
                            elif cls_name == 'McLaren' and conf < 0.55:
                                if old_info['classified']:
                                    cls_name = old_cls
                            # Williams filtering
                            elif cls_name == 'Williams':
                                if old_info['classified'] and old_cls != 'Williams':
                                    cls_name = old_cls
                        break
                
                # If no match found, create new ID
                if object_id is None:
                    object_id = id_counter
                    id_counter += 1
                    track_history[object_id] = []
                    class_history[object_id] = []
                    track_age[object_id] = 0
                else:
                    # Update track age for existing objects
                    track_age[object_id] += 1
                
                # If track is new, be more strict with problematic classes
                if track_age[object_id] < 3 and (cls_name == 'Alpine' or cls_name == 'Red Bull') and conf < 0.75:
                    classified = False
                    cls_name = "F1 Car"  # Use generic for early frames
                
                # Update classification history for stability
                if object_id in class_history:
                    if classified:
                        class_history[object_id].append(cls_name)
                        # Keep history limited
                        if len(class_history[object_id]) > 5:
                            class_history[object_id].pop(0)
                    
                    # Use most common class from history - increased from 3 to 5 for better stability
                    if len(class_history[object_id]) >= 5:
                        counts = Counter(class_history[object_id])
                        if counts:
                            most_common = counts.most_common(1)[0][0]
                            cls_name = most_common
                            classified = True
                
                # Save current detection
                current_detections[object_id] = {
                    'box': (x1, y1, x2, y2),
                    'conf': conf,
                    'class': cls_name,
                    'classified': classified,
                    'center': (center_x, center_y),
                    'area': area,
                    'y_bottom': y2
                }
                
                # Add to detection list
                detections.append({
                    'id': object_id,
                    'box': (x1, y1, x2, y2),
                    'class': cls_name,
                    'classified': classified,
                    'conf': conf,
                    'y_bottom': y2
                })
            
            # Sort by vertical position (cars more below first - closer to camera)
            detections = sorted(detections, key=lambda x: x['y_bottom'], reverse=True)
            
            # Eliminate ghost detections
            detections_to_keep = []
            for det in detections:
                # If it's Alpine, verify if there are other cars very close (possible false positive)
                if det['class'] == 'Alpine' and det['classified']:
                    is_ghost = False
                    
                    for other in detections:
                        if other['id'] != det['id']:
                            # Calculate simple overlap between boxes
                            x1a, y1a, x2a, y2a = det['box']
                            x1b, y1b, x2b, y2b = other['box']
                            
                            overlap = max(0, min(x2a, x2b) - max(x1a, x1b)) * max(0, min(y2a, y2b) - max(y1a, y1b))
                            area_a = (x2a - x1a) * (y2a - y1a)
                            
                            # If there's significant overlap, likely a false positive
                            if overlap > 0.3 * area_a:
                                is_ghost = True
                                break
                    
                    if not is_ghost:
                        detections_to_keep.append(det)
                else:
                    detections_to_keep.append(det)
                    
            detections = detections_to_keep
            
            # Calculate gaps between consecutive cars
            frame_gaps = []
            
            # Process detections for visualization
            if show_video:
                for i, det in enumerate(detections):
                    x1, y1, x2, y2 = det['box']
                    cls_name = det['class']
                    conf = det['conf']
                    classified = det['classified']
                    
                    # Get specific color for the team
                    if classified:
                        color = class_colors.get(cls_name, (0, 255, 0))
                    else:
                        color = class_colors['unknown']  # Yellow for cars without secure classification
                    
                    # Draw box with team color
                    cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                    
                    # Label with class and confidence
                    if classified:
                        label = f"{cls_name}: {conf:.2f}"
                    else:
                        label = f"F1 Car: {conf:.2f}"
                    
                    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    cv2.rectangle(frame_resized, (x1, y1-t_size[1]-3), (x1+t_size[0], y1), color, -1)
                    cv2.putText(frame_resized, label, (x1, y1-3), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    
                    # Only if there's a next car
                    if i < len(detections)-1:
                        next_det = detections[i+1]
                        gap_info = calculate_gap(
                            det['box'], next_det['box'], 
                            det['class'] if det['classified'] else "F1 Car", 
                            next_det['class'] if next_det['classified'] else "F1 Car"
                        )
                        
                        # Save gap info for data collection if this is a sample frame
                        if is_sample_frame:
                            frame_gaps.append({
                                'frame': current_frame,
                                'timestamp': timestamp,
                                'car1_id': det['id'],
                                'car2_id': next_det['id'],
                                'car1_team': gap_info['car1'],
                                'car2_team': gap_info['car2'],
                                'distance_meters': gap_info['distance'],
                                'gap_seconds': gap_info['time']
                            })
                        
                        # Connection points
                        cx1, cy1 = int((x1+x2)/2), int(y1)  # Use top of the car
                        nx1, ny1, nx2, ny2 = next_det['box']
                        cx2, cy2 = int((nx1+nx2)/2), int(ny2)  # Use bottom of the next car
                        
                        # Diagonal line between cars
                        line_color = (0, 0, 255) if is_sample_frame else (0, 255, 0)  # Red if sampled, otherwise green
                        line_thickness = 3 if is_sample_frame else 2  # Thicker if sampled
                        cv2.line(frame_resized, (cx1, cy1), (cx2, cy2), line_color, line_thickness)
                        
                        # Text at midpoint with more information
                        mid_x, mid_y = (cx1+cx2)//2, (cy1+cy2)//2
                        
                        # Distance and gap time
                        dist_text = f"{gap_info['distance']:.1f}m"
                        time_text = f"{gap_info['time']:.2f}s"
                        
                        # Background for text
                        dist_size = cv2.getTextSize(dist_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                        time_size = cv2.getTextSize(time_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                        
                        # Draw semi-transparent background
                        overlay = frame_resized.copy()
                        bg_color = (0, 0, 180) if is_sample_frame else (0, 0, 0)  # Dark red if sampled
                        cv2.rectangle(overlay, 
                                    (mid_x - 5, mid_y - 50), 
                                    (mid_x + max(dist_size[0], time_size[0]) + 10, mid_y + 10),
                                    bg_color, -1)
                        cv2.addWeighted(overlay, 0.6, frame_resized, 0.4, 0, frame_resized)
                        
                        # Draw texts - add "SAVED" mark if sampled
                        if is_sample_frame:
                            saved_text = "SAVED"
                            cv2.putText(frame_resized, saved_text, (mid_x, mid_y - 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                        
                        # Draw gap measurements
                        cv2.putText(frame_resized, dist_text, (mid_x, mid_y - 25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                        cv2.putText(frame_resized, time_text, (mid_x, mid_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # If processing a sample frame, also collect gap data without visualization
            elif is_sample_frame:
                for i, det in enumerate(detections):
                    if i < len(detections)-1:
                        next_det = detections[i+1]
                        gap_info = calculate_gap(
                            det['box'], next_det['box'], 
                            det['class'] if det['classified'] else "F1 Car", 
                            next_det['class'] if next_det['classified'] else "F1 Car"
                        )
                        
                        frame_gaps.append({
                            'frame': current_frame,
                            'timestamp': round(timestamp, 2),
                            'car1_id': det['id'],
                            'car2_id': next_det['id'],
                            'car1_team': gap_info['car1'],
                            'car2_team': gap_info['car2'],
                            'distance_meters': round(gap_info['distance'], 2),
                            'gap_seconds': round(gap_info['time'], 2)
                        })
            
            # Add all gaps from this frame to our collection if it's a sample frame
            if is_sample_frame:
                all_gaps.extend(frame_gaps)
                print(f"Found {len(frame_gaps)} car gaps at timestamp {timestamp:.2f}s")
        
        # Update last_detections for next iteration
        last_detections = current_detections
        
        # Calculate FPS for display
        if show_video and frame_count % 10 == 0:
            current_time = pytime.time()
            current_fps = 10.0 / (current_time - start_time)
            start_time = current_time
        
        # Show visual elements if requested
        if show_video:
            # Show FPS and model information
            cv2.putText(frame_resized, f"FPS: {current_fps:.1f}", (20, 40),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Add "SAMPLE FRAME" indicator if this is a frame being saved
            if is_sample_frame:
                cv2.putText(frame_resized, "▶ SAMPLE FRAME", (FRAME_WIDTH - 300, 40),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame_resized, "F1 Gap Extraction", (FRAME_WIDTH - 300, 40),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Show detection threshold
            detection_mode = f"Detection Threshold: {GAP_DETECTION_THRESHOLD:.2f}"
            cv2.putText(frame_resized, detection_mode, (20, target_height - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Show progress
            progress_text = f"Frame: {current_frame}/{total_frames} ({current_frame/total_frames*100:.1f}%)"
            cv2.putText(frame_resized, progress_text, (FRAME_WIDTH - 400, target_height - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Show the frame
            cv2.imshow("F1 Gap Extraction", frame_resized)
            
            # Handle key presses
            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            elif key == ord('+'):  # Increase threshold
                GAP_DETECTION_THRESHOLD = min(GAP_DETECTION_THRESHOLD + 0.05, 0.95)
                print(f"Detection threshold increased to {GAP_DETECTION_THRESHOLD:.2f}")
            elif key == ord('-'):  # Decrease threshold
                GAP_DETECTION_THRESHOLD = max(GAP_DETECTION_THRESHOLD - 0.05, 0.05)
                print(f"Detection threshold decreased to {GAP_DETECTION_THRESHOLD:.2f}")
            elif key == ord('d'):  # Skip forward 10 seconds
                skip_frames = int(fps * 10)
                new_frame_pos = min(current_frame + skip_frames, total_frames - 1)
                cap.set(cv2.CAP_PROP_POS_FRAMES, new_frame_pos)
                current_frame = new_frame_pos - 1
                last_detections = {}
                print(f"Skipped forward 10 seconds to frame {new_frame_pos}")
    
    # Release video resource
    cap.release()
    cv2.destroyAllWindows()
    
    # Convert to DataFrame
    if all_gaps:
        gaps_df = pd.DataFrame(all_gaps)
        
        # Save to CSV if requested
        if output_csv is None:
            # Generate default filename based on video name
            video_name = os.path.basename(video_path).split('.')[0]
            timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_csv = f"../f1-strategy/data/gaps/gap_data_{video_name}_{timestamp_str}.csv"
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(output_csv), exist_ok=True)
        
        # Save data
        gaps_df.to_csv(output_csv, index=False, float_format='%.3f')
        print(f"Gap data saved to {output_csv}")
        print(f"Total of {len(gaps_df)} gap measurements collected")
        
        return gaps_df
    else:
        print("No gap data could be collected!")
        return None

In [53]:
# Cell 3: Run gap extraction with visualization
def main_extract_gaps():
    """Extract gaps from the video at 10-second intervals and save to CSV"""
    video_path = "videos/belgium_gp.f399.mp4"
    
    # Run extraction with default 10-second interval, showing video
    gap_data = extract_gaps_from_video(
        video_path,
        sample_interval_seconds=10,
        show_video=True  # Enable video display
    )
    
    # Display sample of extracted data
    if gap_data is not None and not gap_data.empty:
        print("\nSample gap data:")
        print(gap_data.head())
        
        # Show some basic statistics
        print("\nGap statistics (seconds):")
        print(gap_data['gap_seconds'].describe())
    
    return gap_data

In [54]:
# # Cell 4: Execute the extraction
if __name__ == "__main__":
     gap_data = main_extract_gaps()

Video FPS: 50, sampling every 500 frames (10 seconds)
Starting gap extraction from videos/belgium_gp.f399.mp4...
Will sample approximately every 10 seconds
Taking sample at frame 1 (timestamp: 0.02s)
Found 1 car gaps at timestamp 0.02s
Skipped forward 10 seconds to frame 529
Taking sample at frame 529 (timestamp: 10.58s)
Found 0 car gaps at timestamp 10.58s
Skipped forward 10 seconds to frame 1038
Taking sample at frame 1038 (timestamp: 20.76s)
Found 2 car gaps at timestamp 20.76s
Skipped forward 10 seconds to frame 1627
Taking sample at frame 1627 (timestamp: 32.54s)
Found 0 car gaps at timestamp 32.54s
Skipped forward 10 seconds to frame 2162
Taking sample at frame 2162 (timestamp: 43.24s)
Skipped forward 10 seconds to frame 2901
Taking sample at frame 2901 (timestamp: 58.02s)
Found 0 car gaps at timestamp 58.02s
Detection threshold decreased to 0.35
Detection threshold decreased to 0.30
Skipped forward 10 seconds to frame 3464
Taking sample at frame 3464 (timestamp: 69.28s)
Found 4 

---

## 3. Conclussions

In this notebook, we've successfully implemented a computer vision system that:

1. Detects F1 cars in video footage using our YOLOv8 model
2. Calculates gaps between consecutive cars in both meters and seconds
3. Extracts data at regular intervals (every 10 seconds) to feed into our strategy system
4. Saves this information to structured CSV files with team identification


### 3.1 CSV Structure

Our gap extraction system produces CSV files with the following columns:

- ``frame``: Frame number in the video
- ``timestamp``: Time in seconds from the start of the video
- ``car1_id and car2_id``: Unique identifiers for each detected car
- ``car1_team and car2_team``: Team identifications based on our model
- ``distance_meters``: Physical distance between cars in meters
- ``gap_seconds``: Time gap between cars (in seconds at 300 km/h)

This data provides the foundation for our gap-based strategy rules, enabling our expert system to make informed decisions about:

- Undercut/overcut opportunities
- Traffic management
- Defensive positioning


### 3.2 Next Steps
With our gap data now available, the next phase is to integrate this information into our expert system by:

- Creating gap-specific rules that detect strategic opportunities
- Combining gap data with our existing tire degradation and lap time models
- Implementing undercut/overcut detection logic based on real-world F1 strategy principles

In the next notebook (``N05_gap_strategy_rules.ipynb``), we'll develop these rules and integrate all components of our F1 strategy system.

However, due to that this data can be not enough or too much erratic, we will do the gap rules with fastf1 data.